<a href="https://colab.research.google.com/github/showravj2-create/student_performance_prediction/blob/main/student_performance_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# =============================
# 🎓 Student Performance Prediction
# Exploratory Data Analysis & Modeling
# =============================

# 📌 Step 1: Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

# =============================
# 📌 Step 2: Load dataset
# =============================
df = pd.read_csv("/content/sample_data/mnist_test.csv")

# Show first rows
display(df.head())

# =============================
# 📌 Step 3: Basic Exploration
# =============================
print("Dataset shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nMissing values:\n", df.isnull().sum())

# Summary stats
display(df.describe())

# =============================
# 📌 Step 4: Exploratory Data Analysis (EDA)
# =============================

# Distribution of final grade
# Note: This visualization is based on the original 'G3' column
# and might not be relevant for the 'mnist_test.csv' dataset.
# You may need to adapt this visualization based on the new dataset's columns.
# plt.figure(figsize=(6,4))
# sns.histplot(df["G3"], bins=15, kde=True, color="blue")
# plt.title("Distribution of Final Grade (G3)")
# plt.show()

# Correlation heatmap
# Note: This visualization is based on the original dataset's numeric columns
# and might not be relevant for the 'mnist_test.csv' dataset.
# You may need to adapt this visualization based on the new dataset's columns.
# plt.figure(figsize=(12,8))
# sns.heatmap(df.corr(numeric_only=True), cmap="coolwarm", annot=False)
# plt.title("Correlation Heatmap")
# plt.show()

# Study time vs Final grade
# Note: This visualization is based on the original 'studytime' and 'G3' columns
# and might not be relevant for the 'mnist_test.csv' dataset.
# You may need to adapt this visualization based on the new dataset's columns.
# plt.figure(figsize=(6,4))
# sns.boxplot(x="studytime", y="G3", data=df)
# plt.title("Study Time vs Final Grade")
# plt.show()


# =============================
# 📌 Step 5: Data Preprocessing
# =============================
# Convert categorical to numeric
df_encoded = pd.get_dummies(df, drop_first=True)

# Note: The original code assumes a target column named 'G3'.
# You will need to identify the target column in 'mnist_test.csv'
# and update the following lines accordingly.
# X = df_encoded.drop("G3", axis=1)
# y = df_encoded["G3"]

# Split dataset
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42
#     )

# Scale features
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# =============================
# 📌 Step 6: Train Models
# =============================

# Note: The following code for model training and evaluation
# is based on the original dataset and target variable 'G3'.
# You will need to adapt this section based on the columns
# and the type of problem (e.g., classification) presented by 'mnist_test.csv'.
# models = {
#     "Linear Regression": LinearRegression(),
#     "Random Forest": RandomForestRegressor(random_state=42),
#     "Support Vector Regressor": SVR()
#     }

# results = {}

# for name, model in models.items():
#     model.fit(X_train, y_train)
#     preds = model.predict(X_test)

#     rmse = np.sqrt(mean_squared_error(y_test, preds))
#     r2 = r2_score(y_test, preds)

#     results[name] = {"RMSE": rmse, "R2": r2}

#     print(f"{name}: RMSE={rmse:.2f}, R²={r2:.2f}")

# # =============================
# # 📌 Step 7: Compare Models
# # =============================
# results_df = pd.DataFrame(results).T
# display(results_df)

# # =============================
# # 📌 Step 8: Visualization of Model Performance
# # =============================
# plt.figure(figsize=(8,5))
# sns.barplot(x=results_df.index, y="R2", data=results_df.reset_index())
# plt.title("Model Comparison (R² Score)")
# plt.ylabel("R² Score")
# plt.xlabel("Model")
# plt.ylim(0,1)
# plt.show()

# # =============================
# # 📌 Step 9: Prediction vs Actual (Best Model)
# # =============================
# best_model = RandomForestRegressor(random_state=42)
# best_model.fit(X_train, y_train)
# preds = best_model.predict(X_test)

# plt.figure(figsize=(6,6))
# plt.scatter(y_test, preds, alpha=0.6, color="blue")
# plt.plot([0,20],[0,20], color="red", linestyle="--")
# plt.xlabel("Actual Grades")
# plt.ylabel("Predicted Grades")
# plt.title("Random Forest: Predicted vs Actual Grades")
# plt.show()

,7,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Dataset shape: (9999, 785)

Columns: ['7', '0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '0.10', '0.11', '0.12', '0.13', '0.14', '0.15', '0.16', '0.17', '0.18', '0.19', '0.20', '0.21', '0.22', '0.23', '0.24', '0.25', '0.26', '0.27', '0.28', '0.29', '0.30', '0.31', '0.32', '0.33', '0.34', '0.35', '0.36', '0.37', '0.38', '0.39', '0.40', '0.41', '0.42', '0.43', '0.44', '0.45', '0.46', '0.47', '0.48', '0.49', '0.50', '0.51', '0.52', '0.53', '0.54', '0.55', '0.56', '0.57', '0.58', '0.59', '0.60', '0.61', '0.62', '0.63', '0.64', '0.65', '0.66', '0.67', '0.68', '0.69', '0.70', '0.71', '0.72', '0.73', '0.74', '0.75', '0.76', '0.77', '0.78', '0.79', '0.80', '0.81', '0.82', '0.83', '0.84', '0.85', '0.86', '0.87', '0.88', '0.89', '0.90', '0.91', '0.92', '0.93', '0.94', '0.95', '0.96', '0.97', '0.98', '0.99', '0.100', '0.101', '0.102', '0.103', '0.104', '0.105', '0.106', '0.107', '0.108', '0.109', '0.110', '0.111', '0.112', '0.113', '0.114', '0.115', '0.116', '0.117', '0.118'

,7,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
count,9999.000000,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,...,9999.000000,9999.000000,9999.000000,9999.000000,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0
mean,4.443144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.179318,0.163616,0.052605,0.000600,0.0,0.0,0.0,0.0,0.0,0.0
std,2.895897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.674433,5.736359,2.420125,0.060003,0.0,0.0,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,253.000000,253.000000,156.000000,6.000000,0.0,0.0,0.0,0.0,0.0,0.0


from matplotlib import pyplot as plt
_df_0['7'].plot(kind='hist', bins=20, title='7')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['0'].plot(kind='hist', bins=20, title='0')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_7.plot(kind='scatter', x='0.2', y='0.3', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_8['7'].plot(kind='line', figsize=(8, 4), title='7')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_9['0'].plot(kind='line', figsize=(8, 4), title='0')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_10['0.1'].plot(kind='line', figsize=(8, 4), title='0.1')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_11['0.2'].plot(kind='line', figsize=(8, 4), title='0.2')
plt.gca().spines[['top', 'right']].set_visible(False)